In [3]:
import tensorflow as tf
import numpy as np
import functools
from tensorflow.python.keras.applications.mobilenet import MobileNet
import tensorflow_hub as hub
import os
#tf.enable_eager_execution()
#tf.logging.set_verbosity(tf.logging.ERROR)

In [4]:
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
def _parse_function(filename, label):
    image_string = tf.read_file(filename)
    image = tf.image.decode_image(image_string, channels=3)
    #image = tf.image.central_crop(image, central_fraction=0.875)
    #image = tf.image.resize_images(image, [224, 224])
    image = inception_preprocessing.preprocess_image(image, 224, 224, is_training=True)
    return image, tf.string_to_number(label, out_type=tf.int32)

In [3]:
def ImagenetTrainInput():
    train_files = tf.constant([s.strip("\n") for s in open("/data3/imagenet/imagenet_train.files").readlines()])
    train_labels = tf.constant([s.strip("\n") for s in open("/data3/imagenet/imagenet_train.labels").readlines()])
    train_dataset = tf.data.Dataset.from_tensor_slices((train_files, train_labels))
    train_dataset = train_dataset.apply(tf.contrib.data.map_and_batch(
    map_func=_parse_function, batch_size=512))
    train_dataset = train_dataset.prefetch(tf.contrib.data.AUTOTUNE)
    return train_dataset

In [4]:
#train_files = tf.constant([s.strip("\n") for s in open("/data3/imagenet/imagenet_train.files").readlines()])
#train_labels = tf.constant([s.strip("\n") for s in open("/data3/imagenet/imagenet_train.labels").readlines()])

#val_files = tf.constant([s.strip("\n") for s in open("/data3/imagenet/imagenet_val.files").readlines()])
#val_labels = tf.constant([s.strip("\n") for s in open("/data3/imagenet/imagenet_val.labels").readlines()])

In [5]:
#train_dataset = tf.data.Dataset.from_tensor_slices((train_files, train_labels))
#val_dataset = tf.data.Dataset.from_tensor_slices((val_files, val_labels))

In [6]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    #input_layer = tf.reshape(features, [-1, 28, 28, 1])
    # Convolutional Layer #1
    # use a tf.tran.scaffold to set the init ops needed to leave pretrained weights
    # alone, otherwise this bad boy overwrites everything immediately
    model = hub.Module("https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/1")
    logits = tf.layers.dense(inputs=model(features), units=1000)

    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    #accuracy = tf.metrics.accuracy(
    #    labels=labels, predictions=tf.argmax(logits, axis=1))
    # can use this for more logging
    #tf.identity('cross_entropy', loss)
    #tf.summary.scalar('train_accuracy', accuracy[1])
    tf.summary.scalar("loss", loss)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        # can use name scope in layer creation then 
        #"scope/prefix/for/first/vars")
        # first_train_op = optimizer.minimize(cost, var_list=first_train_vars)
        # can combine multiple train ops with tf.group(op1, op2)
        # print the sum of losses and use tensorboard to visualize each
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [7]:
classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="/data/jwfromm/models/pretrained_net")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/data/jwfromm/models/pretrained_net', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7bd1cf59b0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
classifier.train(input_fn=ImagenetTrainInput, steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Initialize variable module/MobilenetV2/Conv/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/e23751975d8304461650459778e453d44e7298d6/variables/variables' with MobilenetV2/Conv/BatchNorm/beta
INFO:tensorflow:Initialize variable module/MobilenetV2/Conv/BatchNorm/gamma:0 from checkpoint b'/tmp/tfhub_modules/e23751975d8304461650459778e453d44e7298d6/variables/variables' with MobilenetV2/Conv/BatchNorm/gamma
INFO:tensorflow:Initialize variable module/MobilenetV2/Conv/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/e23751975d8304461650459778e453d44e7298d6/variables/variables' with MobilenetV2/Conv/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/MobilenetV2/Conv/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/e23751975d8304461650459778e453d44e7298d6/variables/variables' with MobilenetV2/Conv/BatchNorm/moving_variance
INFO:tensorflow:I